# 1. Импортирование библиотек и парсинг данных

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import re
import os
from glob import glob
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models, transforms
from tqdm import tqdm
# Отключаем проверку
import warnings
warnings.filterwarnings("ignore")

In [166]:
list_bmp = []
list_frame = []
def parsing_bmp(dir):
    list = os.listdir(dir)    
    for item in glob(dir+'\\'+'*.bmp'):
        list_bmp.append(item)
    for item in glob(dir+'\\'+'*.frame'):
        list_frame.append(item)
    for dir_for in list:
        if os.path.isdir(dir+'\\'+dir_for):
            parsing_bmp(dir+'\\'+dir_for)
parsing_bmp('.')
list_bmp

['.\\DATAFRAME\\FRAMES\\frame0000.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0000.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0001.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0002.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0003.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0004.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0005.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0006.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0007.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0008.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0009.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0010.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0011.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0012.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0000.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0001.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0002.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0003.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0004.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0005.bmp',
 '.\\DATAFRAME\\FRAMES\\0\\15

In [167]:
list_frame

['.\\DATAFRAME\\FRAMES\\frame0000.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0000.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0001.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0002.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0003.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0004.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0005.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0006.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0007.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0008.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0009.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0010.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0011.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1530\\frame0012.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0000.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0001.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0002.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0003.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame0004.frame',
 '.\\DATAFRAME\\FRAMES\\0\\1538\\frame00

In [168]:
print(f'Файлов формата bmp {len(list_bmp)} и формата frame {len(list_frame)}')

Файлов формата bmp 7808 и формата frame 7808


# 2. Чтение изображения

In [172]:
img = cv2.imread(list_bmp[0], cv2.IMREAD_COLOR)